In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
DEVICE

In [ ]:
if DEVICE.type == 'cuda' :
    !nvidia-smi

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

house = datasets.load_boston()
print(house.DESCR)

In [ ]:
x_data = house.data
y_data = house.target
print(x_data.shape, y_data.shape)   # (506, 13) (506,)

import pandas as pd
df = pd.DataFrame(house.data, columns=house.feature_names)
df['target'] = house.target
print(df.head())

In [ ]:
#@title 
x_scaler = MinMaxScaler()
x_scaler.fit_transform(x_data)
x_data_scaled = x_scaler.transform(x_data)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, shuffle=True)

print(x_train.shape, y_train.shape) # (404, 13) (404,)
print(x_test.shape, y_test.shape)   # (102, 13) (102,)

x_train, x_test, y_train, y_test = torch.FloatTensor(x_train), torch.FloatTensor(x_test), torch.FloatTensor(y_train).view(-1, 1), torch.FloatTensor(y_test).view(-1, 1)

print(x_train.shape, y_train.shape) # (404, 13) (404,)
print(x_test.shape, y_test.shape)   # (102, 13) (102,)

In [ ]:
class Net(nn.Module):
    def __init__(self, dim_input):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(dim_input, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 1)
        
        self.activation = torch.nn.ReLU()        
        
                      
    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)  # x = F.relu(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.fc4(x)
        x = self.activation(x)
        x = self.fc5(x)

        return x

In [ ]:
model = Net(x_train.shape[1])
criterion = torch.nn.MSELoss().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=1e-5)

import time

nb_epochs = 5000
start_time = time.time()
for epoch in range(nb_epochs + 1):    # H(x) 계산
    prediction = model(x_train)
    cost = criterion(prediction, y_train)  

    optimizer.zero_grad()
    cost.backward()  # gradient 계산
    optimizer.step()  # update

    # 200번마다 로그 출력
    if epoch % 200 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

print()
print("Time  : ", time.time()-start_time)
print("X     : ", x_test[0])
print("Y     : ", y_test[0])
print("Answer: ", model(x_test).data[0])